# Create Multiple Point Defects in a Bulk Material

Create a vacancy, add substitution or interstitial atom to provided Material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook. 
1. Set notebook parameters in cell 1.1. below (or use the default values) -- to create multiple defects, fill `DEFECT_CONFIGS` with a list of dictionaries. 
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

In [ ]:
from types import SimpleNamespace

# Selected material will be used as a unit cell to create a supercell first.
SUPERCELL_MATRIX = [[3, 0, 0], [0, 3, 0], [0, 0, 3]]

DEFECT_CONFIGS = [
    {
        "type": "substitution",
        "coordinate": [0.0, 0.0, 0.0],  # Crystal coordinates
        "element": "C",
        "placement_method": "closest_site",
    },

    {
        "type": "vacancy",
        "coordinate": [0.25, 0.25, 0.25],  # Crystal coordinates
        "placement_method": "closest_site",
    },

    {
        "type": "interstitial",
        "coordinate": [0.75, 0.75, 0.75],  # Crystal coordinates
        "element": "N",
        "placement_method": "voronoi_site",
    },
]

DEFECT_CONFIGS = [SimpleNamespace(**defect_config) for defect_config in DEFECT_CONFIGS]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.helpers import create_supercell

unit_cell = materials[0]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create Multiple Defects
### 2.1. Prepare defect dictionaries and validate them

In [ ]:
from mat3ra.made.tools.helpers import PointDefectDict
defect_dicts = [PointDefectDict(**defect_dict.__dict__) for defect_dict in DEFECT_CONFIGS]

print("Defect configurations:")
for i, defect_dict in enumerate(DEFECT_CONFIGS):
    print(f"\n{i+1}. {defect_dict.type.capitalize()} defect:")
    print(f"   - Coordinate: {defect_dict.coordinate}")
    print(f"   - Resolution method: {defect_dict.placement_method}")
    if hasattr(defect_dict, "element"):
        print(f"   - Element: {defect_dict.element}")
    
print(f"\nTotal defects to create: {len(DEFECT_CONFIGS)}")

### 2.2. Create multiple defects using the newer approach

In [ ]:
from mat3ra.made.tools.helpers import create_multiple_defects

material_with_defects = create_multiple_defects(
    material=supercell,
    defect_dicts=defect_dicts,
)

print(f"Original atoms: {len(supercell.basis.elements.ids)}")
print(f"Final atoms: {len(material_with_defects.basis.elements.ids)}")

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defects, "title": f"Material with defects"}],
          rotation="-90x")
visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defects, "title": f"Material with defects"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

# Set the material name and pass to outside runtime
material_with_defects.name = f"{unit_cell.name} with {len(DEFECT_CONFIGS)} defects"
set_materials([material_with_defects])